In [1]:
from os.path import join

from keras.applications import VGG16, InceptionV3
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model, load_model
from keras.utils.np_utils import to_categorical

import pandas as pd
import csv
import os
import numpy as np
import json

from matplotlib import pyplot as plt
import sys
sys.path.append("../data_preparation/")

from batch_generator import BatchGenerator, BatchSequence

from sklearn.metrics import recall_score, precision_score, f1_score

/scratch/sekiz/FashionChallenge/.env/local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## data folder

In [2]:
#datadir = os.getcwd()
input_path = os.path.abspath('../../mlipdata/')

train={}
test={}
validation={}
with open(os.path.join(input_path, 'train.json')) as json_data:
    train= json.load(json_data)
with open(os.path.join(input_path, 'test.json')) as json_data:
    test= json.load(json_data)
with open(os.path.join(input_path, 'validation.json')) as json_data:
    validation = json.load(json_data)

print('Train No. of images: %d'%(len(train['images'])))
print('Test No. of images: %d'%(len(test['images'])))
print('Validation No. of images: %d'%(len(validation['images'])))

# JSON TO PANDAS DATAFRAME
# train data
train_img_url=train['images']
train_img_url=pd.DataFrame(train_img_url)
train_ann=train['annotations']
train_ann=pd.DataFrame(train_ann)
train=pd.merge(train_img_url, train_ann, on='imageId', how='inner')

# test data
test=pd.DataFrame(test['images'])

# Validation Data
val_img_url=validation['images']
val_img_url=pd.DataFrame(val_img_url)
val_ann=validation['annotations']
val_ann=pd.DataFrame(val_ann)
validation=pd.merge(val_img_url, val_ann, on='imageId', how='inner')

datas = {'Train': train, 'Test': test, 'Validation': validation}
for data in datas.values():
    data['imageId'] = data['imageId'].astype(np.uint32)

In [4]:
# np.array(train.labelId)
images_path_train = os.path.abspath('../../mlipdata/files/train/')
images_path_validation = os.path.abspath('../../mlipdata/files/validation/')
images_path_test = os.path.abspath('../../mlipdata/files/test/')

# testing pretrained

# VGG16

In [ ]:
y_test = np.zeros((39706,1))
predict_gen = BatchSequence(input_dir=images_path_test, y=y_test, batch_size=128)
modelvgg16 = load_model('../models/VGG16-finetuned-1_epochs.h5')
predictions = modelvgg16.predict_generator(predict_gen, verbose=1)

120/311 [==========>...................] - ETA: 1:18:23

In [118]:
threshold = 0
y_pred = (predictions > threshold)

results =[ " ".join([str(i-1) for i in j if i !=0]) for j in y_pred*range(1,229)]
results = pd.Series(results, name='label_id')
submission = pd.concat([pd.Series(range(1,y_pred.shape[0]+1), name='image_id', dtype=object), results], axis=1)
submission.to_csv("vgg16.csv", index=False, quoting=csv.QUOTE_NONE)

# VGG19

In [5]:
y_test = np.zeros((39706,1))
predict_gen = BatchSequence(input_dir=images_path_test, y=y_test, batch_size=128)
modelvgg19 = load_model('../models/VGG19-finetuned-1_epochs.h5')
predictions = modelvgg19.predict_generator(predict_gen, verbose=1)

In [ ]:
threshold = 0
y_pred = (predictions > threshold)

results =[ " ".join([str(i-1) for i in j if i !=0]) for j in y_pred*range(1,229)]
results = pd.Series(results, name='label_id')
submission = pd.concat([pd.Series(range(1,y_pred.shape[0]+1), name='image_id', dtype=object), results], axis=1)
submission.to_csv("vgg19.csv", index=False, quoting=csv.QUOTE_NONE)

In [ ]:
y_train = np.array([j[1:] for j in y_train])


# Testing

In [10]:
def multiple_batch_generator(generator_num, **kwargs):
    """A generator to work with multiple inputs models
    
    We create a model with a list of multiple input layers when 
    we use :func:`keras.layers.concatenate`. However,
    :class:`batch_generator.BatchGenerator` returns a single tuple 
    with two arrays, which does not fit to a model with a multiple 
    input layers. Thus, with this generator, we create the necessary 
    input for such models.
    
    Arguments:
        generator_num {int} -- number of generators should be created
        \**kwargs -- See :class:`batch_generator.BatchGenerator`
        
    Yields:
        ([ndarray,...,ndarray], ndarray) -- in the tuple; list contains feature arrays from each generator, array out of the list contains the label set
    """
    generators_list = [BatchGenerator(**kwargs) for i in range(generator_num)]
    
    while True:
        Xy_list = [gen.next() for gen in generators_list]
        yield [Xy[0] for Xy in Xy_list], Xy_list[0][1]

In [ ]:
y_test = np.zeros((39706,1))
batch_size = 50
test_steps = int(len(y_test)/batch_size)
input_num = len(model.input_layers)

In [ ]:
test_generator_multi = multiple_batch_generator(generator_num=input_num,
                                               input_dir=images_path_test,
                                               y=y_test,
                                               batch_size=batch_size)

In [ ]:
predictions = stackedmodel.predict_generator(test_generator_multi, steps=test_steps, verbose=1)

In [7]:
threshold = 0
y_pred = (predictions > threshold)

results =[ " ".join([str(i-1) for i in j if i !=0]) for j in y_pred*range(1,229)]
results = pd.Series(results, name='label_id')
submission = pd.concat([pd.Series(range(1,y_pred.shape[0]+1), name='image_id', dtype=object), results], axis=1)
submission.to_csv("stacked.csv", index=False, quoting=csv.QUOTE_NONE)

NameError: name 'predictions' is not defined

#### predict_generator with multiple inputs

In [43]:
batch_size = 50
val_steps = int(len(y_validation[:500])/batch_size)

In [44]:
input_num = len(model.input_layers)

In [45]:
val_generator_multi = multiple_batch_generator(generator_num=input_num,
                                               input_dir=images_path_validation,
                                               y=y_validation
                                               [:500],
                                               batch_size=batch_size)

In [38]:
stackedmodel = load_model('..')

# Validation

In [39]:
# Set the paths
input_path = os.path.abspath('../../mlipdata/')

with open('pickles/binarizer.pickle', 'rb') as pickle_file:
    binarizer = pickle.load(pickle_file)

In [40]:
images_path_validation = os.path.join(input_path, 'files/validation/')

In [41]:
with file_io.FileIO('../../mlipdata/server_validation.pickle', mode='rb') as fp:
        data = gzip.GzipFile(fileobj=fp)
        y_validation = cPickle.load(data)
y_validation = np.array([j[1:] for j in y_validation])     

In [42]:
validation={}
with open(os.path.join(input_path, 'validation.json')) as json_data:
    validation = json.load(json_data)

validation_img_url = validation['images']
validation_img_url = pd.DataFrame(validation_img_url)
validation_ann = validation['annotations']
validation_ann = pd.DataFrame(validation_ann)
validation = pd.merge(validation_img_url, validation_ann, on='imageId', how='inner')
validation['imageId'] = validation['imageId'].astype(np.uint32)

#y_validation = np.array(validation.labelId)
#y_validation_bin = binarizer.transform(y_validation)

del validation_img_url
del validation_ann
del validation

#### predict_generator with multiple inputs

In [43]:
batch_size = 50
val_steps = int(len(y_validation[:500])/batch_size)

In [44]:
input_num = len(model.input_layers)

In [45]:
val_generator_multi = multiple_batch_generator(generator_num=input_num,
                                               input_dir=images_path_validation,
                                               y=y_validation
                                               [:500],
                                               batch_size=batch_size)

In [46]:
predictions = model.predict_generator(val_generator_multi, steps=val_steps, verbose=1)

10/10 [==============================] - 242s 24s/step


In [47]:
len(predictions)

500

In [48]:
y_true = y_validation[:500]
y_pred = (predictions > 0.5).astype(int)

In [49]:
pr = precision_score(y_true, y_pred, average='micro')
rc = recall_score(y_true, y_pred, average='micro')
f1 = f1_score(y_true, y_pred, average='micro')

print("Precision: {} Recall: {} F1: {}".format(pr, rc, f1))

Precision: 0.0715523392111 Recall: 0.181536074476 F1: 0.102646585758


In [46]:
predictions = model.predict_generator(val_generator_multi, steps=val_steps, verbose=1)

10/10 [==============================] - 242s 24s/step


In [48]:
y_true = y_validation[:500]
y_pred = (predictions > 0.5).astype(int)

In [49]:
pr = precision_score(y_true, y_pred, average='micro')
rc = recall_score(y_true, y_pred, average='micro')
f1 = f1_score(y_true, y_pred, average='micro')

print("Precision: {} Recall: {} F1: {}".format(pr, rc, f1))

Precision: 0.0715523392111 Recall: 0.181536074476 F1: 0.102646585758


500

ValueError: Error when checking model : the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 4 array(s), but instead got the following list of 1 arrays: [array([[[[146, 143, 138],
         [202, 173, 159],
         [185, 147, 124],
         ...,
         [197, 182, 149],
         [198, 187, 151],
         [192, 185, 148]],

        [[147, 144, 137],
 ...